In [15]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import importlib
from scipy.stats import zscore
sys.stdout.flush()
np.random.seed(0)

In [18]:
#"ordinary" lightcurves
rho_file=np.loadtxt("synthetic_rhos_v2.csv", delimiter=',')
#"outlier" lightcurves
sine_file=np.loadtxt("synthetic_sines_v3.csv", delimiter=',')

rho_train, rho_valid, rho_train_ids, rho_valid_ids= train_test_split(rho_file, list(range(len(rho_file))) ,test_size=0.25, random_state=0)

k_clusters=[10, 50, 100, 200]
seg_lens=[10, 50, 100,150,200]

# Approach I

1. Segment ordinary time series. 
2. Cluster the segments. 
3. Segment the validation time series of *ordinary* class and try to reconstruct using cluster centroids. Calculate the Chi-square, divided by the number of reconstructed data points, for each time series.
4. Repeat 3. for validation time series of the *outlier* class. 

In [19]:
import segment_cluster as sc
importlib.reload(sc)

reco_error=[]

for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        ##train the model
        #loop throught the light curves of a given class and segments them
        all_train_segments=[]
        for rho in rho_train:
            train_segments=sc.segmentation(rho, seg_len, seg_len, time_stamps=False)
            all_train_segments.append(train_segments)
        all_train_segments=np.vstack(all_train_segments)
        #cluster the segments
        cluster=KMeans(n_clusters=k_cluster, random_state=0)
        cluster.fit(all_train_segments)     
        

        ### reconstruction of the training class
        for n_rho, rho in enumerate(rho_valid):
            valid_segments= sc.segmentation(rho, seg_len, seg_len , time_stamps=False)
            reco= sc.reconstruct(valid_segments, rho, cluster, rel_offset=False, seg_slide=seg_len)
            
            reco[0:-seg_len]=reco[0:-seg_len]
            rho_expected=np.copy(rho[0:-seg_len])
            rho_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-rho_expected)/rho_error)**2.0)
            reco_error.append((k_id,len_id,0, rho_valid_ids[n_rho], error))
            print((k_id,len_id,0, rho_valid_ids[n_rho], error), flush=True)


        #reconstruction loop through light curves for every class other than rho              
        for n_sine, sine in enumerate(sine_file):
            valid_segments= sc.segmentation(sine, seg_len, seg_len , time_stamps=False)
            reco = sc.reconstruct(valid_segments, sine, cluster, rel_offset=False, seg_slide=seg_len)
            
            #reco[0:-seg_len] = np.mean(sine[0:-seg_len])+ (reco[0:-seg_len]- np.mean(reco[0:-seg_len]))*(np.std(sine[0:-seg_len])/np.std(reco[0:-seg_len]))
            reco[0:-seg_len]=(reco[0:-seg_len])
            sine_expected=np.copy((sine[0:-seg_len]))
            sine_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-sine_expected)/sine_error)**2.0)
            reco_error.append((k_id,len_id,1,n_sine, error))
            print((k_id,len_id,1,n_sine, error), flush=True)
reco_error_ar=np.array(reco_error)
np.savetxt("chi2_20190610_1.csv", reco_error_ar, delimiter=",") 

(0, 0, 0, 993, 8.669620445575362)
(0, 0, 0, 859, 6.490498403904081)
(0, 0, 0, 298, 15.270733150208995)
(0, 0, 0, 553, 7.400031895318245)
(0, 0, 0, 672, 15.416979352203686)
(0, 0, 0, 971, 11.889960968715373)
(0, 0, 0, 27, 9.698067067572337)
(0, 0, 0, 231, 14.950551733940424)
(0, 0, 0, 306, 7.957097443133291)
(0, 0, 0, 706, 6.470252181907451)
(0, 0, 0, 496, 7.943774060938908)
(0, 0, 0, 558, 17.559052079927003)
(0, 0, 0, 784, 11.719995971291906)
(0, 0, 0, 239, 8.150203335526985)
(0, 0, 0, 578, 11.00721172526459)
(0, 0, 0, 55, 25.728719055000493)
(0, 0, 0, 906, 18.91683730226061)
(0, 0, 0, 175, 11.03561949825206)
(0, 0, 0, 14, 23.5801997762388)
(0, 0, 0, 77, 13.454648629854757)
(0, 0, 0, 31, 10.340362947986714)
(0, 0, 0, 481, 7.837696723533657)
(0, 0, 0, 310, 8.151271978499478)
(0, 0, 0, 311, 7.885601360196152)
(0, 0, 0, 883, 7.086581223841793)
(0, 0, 0, 788, 13.23253415515881)
(0, 0, 0, 45, 6.830272713677459)
(0, 0, 0, 103, 22.08790840461756)
(0, 0, 0, 760, 49.3175157169337)
(0, 0, 0, 1, 

# Approach II

1. Segment ordinary time series. 
2. Cluster the segments. 
3. Segment the validation time series of *ordinary* class and try to reconstruct using cluster centroids **scaled and shifted to the deviation and mean of each segment**. Calculate the Chi-square, divided by the number of reconstructed data points, for each time series.
4. Repeat 3. for validation time series of the *outlier* class. 

In [9]:
import segment_cluster_scaled as sc
importlib.reload(sc)

reco_error=[]

for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        ##train the model
        #loop throught the light curves of a given class and segments them
        all_train_segments=[]
        for rho in rho_train:
            train_segments=sc.segmentation(rho, seg_len, seg_len, time_stamps=False)
            all_train_segments.append(train_segments)
        all_train_segments=np.vstack(all_train_segments)
        #cluster the segments
        cluster=KMeans(n_clusters=k_cluster, random_state=0)
        cluster.fit(all_train_segments)     
        

        ### reconstruction of the training class
        for n_rho, rho in enumerate(rho_valid):
            valid_segments= sc.segmentation(rho, seg_len, seg_len , time_stamps=False)
            reco= sc.reconstruct(valid_segments, rho, cluster, rel_offset=False, seg_slide=seg_len)
            
            reco[0:-seg_len]=reco[0:-seg_len]
            rho_expected=np.copy(rho[0:-seg_len])
            rho_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-rho_expected)/rho_error)**2.0)
            reco_error.append((k_id,len_id,0, rho_valid_ids[n_rho], error))
            print((k_id,len_id,0, rho_valid_ids[n_rho], error), flush=True)


        #reconstruction loop through light curves for every class other than rho              
        for n_sine, sine in enumerate(sine_file):
            valid_segments= sc.segmentation(sine, seg_len, seg_len , time_stamps=False)
            reco = sc.reconstruct(valid_segments, sine, cluster, rel_offset=False, seg_slide=seg_len)
            
            #reco[0:-seg_len] = np.mean(sine[0:-seg_len])+ (reco[0:-seg_len]- np.mean(reco[0:-seg_len]))*(np.std(sine[0:-seg_len])/np.std(reco[0:-seg_len]))
            reco[0:-seg_len]=(reco[0:-seg_len])
            sine_expected=np.copy((sine[0:-seg_len]))
            sine_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-sine_expected)/sine_error)**2.0)
            reco_error.append((k_id,len_id,1,n_sine, error))
            print((k_id,len_id,1,n_sine, error), flush=True)
reco_error_ar=np.array(reco_error)
np.savetxt("chi2_20190610_2.csv", reco_error_ar, delimiter=",") 

(0, 0, 0, 993, 7.309366479473718)
(0, 0, 0, 859, 8.887846727379255)
(0, 0, 0, 298, 18.250856038063514)
(0, 0, 0, 553, 7.1545950488694565)
(0, 0, 0, 672, 20.033710099707935)
(0, 0, 0, 971, 13.694816770813658)
(0, 0, 0, 27, 10.28441858248411)
(0, 0, 0, 231, 18.08310412596876)
(0, 0, 0, 306, 12.371302291163081)
(0, 0, 0, 706, 10.927954173855925)
(0, 0, 0, 496, 8.371235428220539)
(0, 0, 0, 558, 34.500623700049275)
(0, 0, 0, 784, 20.22827112170011)
(0, 0, 0, 239, 15.265246162282603)
(0, 0, 0, 578, 14.780525642521813)
(0, 0, 0, 55, 11.165503146476283)
(0, 0, 0, 906, 23.6331179784402)
(0, 0, 0, 175, 11.302123892439075)
(0, 0, 0, 14, 32.93859062720433)
(0, 0, 0, 77, 10.51361144198753)
(0, 0, 0, 31, 6.105841537373531)
(0, 0, 0, 481, 8.821565328742702)
(0, 0, 0, 310, 11.640757828777945)
(0, 0, 0, 311, 6.063491562113697)
(0, 0, 0, 883, 8.950691619657153)
(0, 0, 0, 788, 15.080703517567894)
(0, 0, 0, 45, 8.78104567925197)
(0, 0, 0, 103, 20.629011219187188)
(0, 0, 0, 760, 38.43274103252341)
(0, 0, 0

# Approach III

1. Segment ordinary time series. 
2. **Standardize** and cluster the segments. 
3. Segment the validation time series of *ordinary* class, **standardize the segments** and try to reconstruct using cluster centroids. **Calculate the error as the mean squared difference between standardized segments and predicted centroids**.
4. Repeat 3. for validation time series of the *outlier* class. 

In [20]:
import segment_cluster_zscore as sc
importlib.reload(sc)

reco_error=[]
for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        ##train the model
        #loop throught the light curves of a given class and segments them
        all_train_segments=[]
        for rho in rho_train:
            train_segments=sc.segmentation(rho, seg_len, seg_len, time_stamps=False)
            all_train_segments.append(train_segments)
        all_train_segments=np.vstack(all_train_segments)
        #cluster the segments
        cluster=KMeans(n_clusters=k_cluster, random_state=0)
        cluster.fit(zscore(all_train_segments))   
        

        ### reconstruction of the training class
        for n_rho, rho in enumerate(rho_valid):
            valid_segments= sc.segmentation(rho, seg_len, seg_len , time_stamps=False)
            reco, error= sc.reconstruct(valid_segments, rho, cluster, rel_offset=False, seg_slide=seg_len)
            reco[0:-seg_len]=reco[0:-seg_len]
            rho_expected=np.copy(rho[0:-seg_len])
            reco_error.append((k_id,len_id,0, rho_valid_ids[n_rho], error))
            print((k_id,len_id,0, rho_valid_ids[n_rho], error), flush=True)


        #reconstruction loop through light curves for every class other than rho              
        for n_sine, sine in enumerate(sine_file):
            valid_segments= sc.segmentation(sine, seg_len, seg_len , time_stamps=False)
            reco, error = sc.reconstruct(valid_segments, sine, cluster, rel_offset=False, seg_slide=seg_len)
            reco[0:-seg_len]=(reco[0:-seg_len])
            sine_expected=np.copy((sine[0:-seg_len]))
            reco_error.append((k_id,len_id,1,n_sine, error))
            print((k_id,len_id,1,n_sine, error), flush=True)
reco_error_ar=np.array(reco_error)
np.savetxt("chi2_20190610_3.csv", reco_error_ar, delimiter=",") 

(0, 0, 0, 993, 0.0911273409072016)
(0, 0, 0, 859, 0.11046598015132626)
(0, 0, 0, 298, 0.14290117305705222)
(0, 0, 0, 553, 0.1036058666038753)
(0, 0, 0, 672, 0.14515205532017264)
(0, 0, 0, 971, 0.1305095048873525)
(0, 0, 0, 27, 0.10608156900464905)
(0, 0, 0, 231, 0.16143901222310897)
(0, 0, 0, 306, 0.09420849011687193)
(0, 0, 0, 706, 0.07799301392928434)
(0, 0, 0, 496, 0.08902264914506297)
(0, 0, 0, 558, 0.21715506762494502)
(0, 0, 0, 784, 0.08700162383782763)
(0, 0, 0, 239, 0.088558316297692)
(0, 0, 0, 578, 0.11425010340883796)
(0, 0, 0, 55, 0.12208212758432745)
(0, 0, 0, 906, 0.15219366579996074)
(0, 0, 0, 175, 0.10314354665735319)
(0, 0, 0, 14, 0.15064858416323823)
(0, 0, 0, 77, 0.15338671538878246)
(0, 0, 0, 31, 0.10610615159398949)
(0, 0, 0, 481, 0.09961981683851197)
(0, 0, 0, 310, 0.09576256022747696)
(0, 0, 0, 311, 0.07646471159320746)
(0, 0, 0, 883, 0.089591549687362)
(0, 0, 0, 788, 0.11115868151342809)
(0, 0, 0, 45, 0.15167465316931433)
(0, 0, 0, 103, 0.1190008976322329)
(0, 0,

# Approach IV

1. Segment ordinary time series. 
2. Cluster the segments. 
3. Segment the validation time series of *ordinary* class and try to reconstruct using cluster centroids. **Scale and shift the predicted centroid to the segment's deviation and mean.** Calculate the Chi-square, divided by the number of reconstructed data points, for each time series.
4. Repeat 3. for validation time series of the *outlier* class. 

In [17]:
import segment_cluster_scaled_4 as sc
importlib.reload(sc)

reco_error=[]

for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        ##train the model
        #loop throught the light curves of a given class and segments them
        all_train_segments=[]
        for rho in rho_train:
            train_segments=sc.segmentation(rho, seg_len, seg_len, time_stamps=False)
            all_train_segments.append(train_segments)
        all_train_segments=np.vstack(all_train_segments)
        #cluster the segments
        cluster=KMeans(n_clusters=k_cluster, random_state=0)
        cluster.fit(all_train_segments)     
        

        ### reconstruction of the training class
        for n_rho, rho in enumerate(rho_valid):
            valid_segments= sc.segmentation(rho, seg_len, seg_len , time_stamps=False)
            reco= sc.reconstruct(valid_segments, rho, cluster, rel_offset=False, seg_slide=seg_len)
            
            reco[0:-seg_len]=reco[0:-seg_len]
            rho_expected=np.copy(rho[0:-seg_len])
            rho_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-rho_expected)/rho_error)**2.0)
            reco_error.append((k_id,len_id,0, rho_valid_ids[n_rho], error))
            print((k_id,len_id,0, rho_valid_ids[n_rho], error), flush=True)


        #reconstruction loop through light curves for every class other than rho              
        for n_sine, sine in enumerate(sine_file):
            valid_segments= sc.segmentation(sine, seg_len, seg_len , time_stamps=False)
            reco = sc.reconstruct(valid_segments, sine, cluster, rel_offset=False, seg_slide=seg_len)
            
            #reco[0:-seg_len] = np.mean(sine[0:-seg_len])+ (reco[0:-seg_len]- np.mean(reco[0:-seg_len]))*(np.std(sine[0:-seg_len])/np.std(reco[0:-seg_len]))
            reco[0:-seg_len]=(reco[0:-seg_len])
            sine_expected=np.copy((sine[0:-seg_len]))
            sine_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-sine_expected)/sine_error)**2.0)
            reco_error.append((k_id,len_id,1,n_sine, error))
            print((k_id,len_id,1,n_sine, error), flush=True)
reco_error_ar=np.array(reco_error)
np.savetxt("chi2_20190610_4.csv", reco_error_ar, delimiter=",") 

(0, 0, 0, 993, 4.457130596330294)
(0, 0, 0, 859, 2.0664606109203305)
(0, 0, 0, 298, 12.485575112746883)
(0, 0, 0, 553, 3.8272507625774175)
(0, 0, 0, 672, 12.07610559244546)
(0, 0, 0, 971, 5.515045768768481)
(0, 0, 0, 27, 4.510647800922021)
(0, 0, 0, 231, 6.704413580906458)
(0, 0, 0, 306, 2.930714180888722)
(0, 0, 0, 706, 2.4763793806783503)
(0, 0, 0, 496, 3.1362823794309915)
(0, 0, 0, 558, 13.773460401784899)
(0, 0, 0, 784, 5.2808665405027355)
(0, 0, 0, 239, 3.962179729204618)
(0, 0, 0, 578, 5.442005451366672)
(0, 0, 0, 55, 8.182305261823663)
(0, 0, 0, 906, 13.376805524756115)
(0, 0, 0, 175, 5.669298314676301)
(0, 0, 0, 14, 17.82233785233363)
(0, 0, 0, 77, 7.079647732615016)
(0, 0, 0, 31, 3.7001667362851483)
(0, 0, 0, 481, 2.057464894544827)
(0, 0, 0, 310, 3.6540986809010194)
(0, 0, 0, 311, 2.3842973516736654)
(0, 0, 0, 883, 3.3338989725839876)
(0, 0, 0, 788, 3.1674325537429273)
(0, 0, 0, 45, 2.3058470636596677)
(0, 0, 0, 103, 12.07266722571687)
(0, 0, 0, 760, 16.805135234006475)
(0, 0

# Approach V

1. Segment ordinary time series. 
2. Cluster the segments. 
3. Segment the validation time series of *ordinary* class and try to reconstruct using cluster centroids **scaled to the deviation of each segment**. Calculate the Chi-square, divided by the number of reconstructed data points, for each time series.
4. Repeat 3. for validation time series of the *outlier* class. 

In [21]:
import segment_cluster_scaled_5 as sc
importlib.reload(sc)

reco_error=[]

for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        ##train the model
        #loop throught the light curves of a given class and segments them
        all_train_segments=[]
        for rho in rho_train:
            train_segments=sc.segmentation(rho, seg_len, seg_len, time_stamps=False)
            all_train_segments.append(train_segments)
        all_train_segments=np.vstack(all_train_segments)
        #cluster the segments
        cluster=KMeans(n_clusters=k_cluster, random_state=0)
        cluster.fit(all_train_segments)     
        

        ### reconstruction of the training class
        for n_rho, rho in enumerate(rho_valid):
            valid_segments= sc.segmentation(rho, seg_len, seg_len , time_stamps=False)
            reco= sc.reconstruct(valid_segments, rho, cluster, rel_offset=False, seg_slide=seg_len)
            
            reco[0:-seg_len]=reco[0:-seg_len]
            rho_expected=np.copy(rho[0:-seg_len])
            rho_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-rho_expected)/rho_error)**2.0)
            reco_error.append((k_id,len_id,0, rho_valid_ids[n_rho], error))
            print((k_id,len_id,0, rho_valid_ids[n_rho], error), flush=True)


        #reconstruction loop through light curves for every class other than rho              
        for n_sine, sine in enumerate(sine_file):
            valid_segments= sc.segmentation(sine, seg_len, seg_len , time_stamps=False)
            reco = sc.reconstruct(valid_segments, sine, cluster, rel_offset=False, seg_slide=seg_len)
            
            #reco[0:-seg_len] = np.mean(sine[0:-seg_len])+ (reco[0:-seg_len]- np.mean(reco[0:-seg_len]))*(np.std(sine[0:-seg_len])/np.std(reco[0:-seg_len]))
            reco[0:-seg_len]=(reco[0:-seg_len])
            sine_expected=np.copy((sine[0:-seg_len]))
            sine_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-sine_expected)/sine_error)**2.0)
            reco_error.append((k_id,len_id,1,n_sine, error))
            print((k_id,len_id,1,n_sine, error), flush=True)
reco_error_ar=np.array(reco_error)
np.savetxt("chi2_20190610_5.csv", reco_error_ar, delimiter=",") 

(0, 0, 0, 993, 21.79739534649478)
(0, 0, 0, 859, 47.92877312969703)
(0, 0, 0, 298, 128.6669872668093)
(0, 0, 0, 553, 52.90220899238153)
(0, 0, 0, 672, 252.06932704496742)
(0, 0, 0, 971, 67.42461162175367)
(0, 0, 0, 27, 49.24376699330588)
(0, 0, 0, 231, 62.416519896131675)
(0, 0, 0, 306, 113.16616975429866)
(0, 0, 0, 706, 36.44010848861854)
(0, 0, 0, 496, 75.79214341244527)
(0, 0, 0, 558, 161.73674104811693)
(0, 0, 0, 784, 57.492218415248395)
(0, 0, 0, 239, 45.58365613762206)
(0, 0, 0, 578, 67.48276950090217)
(0, 0, 0, 55, 39.56099473953252)
(0, 0, 0, 906, 95.34183041112543)
(0, 0, 0, 175, 20.82389654702518)
(0, 0, 0, 14, 34.13331242568176)
(0, 0, 0, 77, 58.925785197100346)
(0, 0, 0, 31, 23.124073104878825)
(0, 0, 0, 481, 38.77028411293765)
(0, 0, 0, 310, 85.54418277987656)
(0, 0, 0, 311, 23.761461704090493)
(0, 0, 0, 883, 32.28451417589953)
(0, 0, 0, 788, 36.83755914578346)
(0, 0, 0, 45, 91.87901250570414)
(0, 0, 0, 103, 144.62139475218981)
(0, 0, 0, 760, 314.9335719437052)
(0, 0, 0, 1

# Approach VI

1. Segment ordinary time series. 
2. Cluster the segments. 
3. Segment the validation time series of *ordinary* class and try to reconstruct using cluster centroids. **Standardize the time series and its reconstruction.** Calculate the mean squared difference between the time series and its reconstruction as the error value.
4. Repeat 3. for validation time series of the *outlier* class. 

In [22]:
import segment_cluster as sc
importlib.reload(sc)

reco_error=[]

for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        ##train the model
        #loop throught the light curves of a given class and segments them
        all_train_segments=[]
        for rho in rho_train:
            train_segments=sc.segmentation(rho, seg_len, seg_len, time_stamps=False)
            all_train_segments.append(train_segments)
        all_train_segments=np.vstack(all_train_segments)
        #cluster the segments
        cluster=KMeans(n_clusters=k_cluster, random_state=0)
        cluster.fit(all_train_segments)     
        

        ### reconstruction of the training class
        for n_rho, rho in enumerate(rho_valid):
            valid_segments= sc.segmentation(rho, seg_len, seg_len , time_stamps=False)
            reco= sc.reconstruct(valid_segments, rho, cluster, rel_offset=False, seg_slide=seg_len)
            
            reco[0:-seg_len]=zscore(reco[0:-seg_len])
            rho_expected=zscore(np.copy(rho[0:-seg_len]))
           # rho_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-rho_expected))**2.0)
            reco_error.append((k_id,len_id,0, rho_valid_ids[n_rho], error))
            print((k_id,len_id,0, rho_valid_ids[n_rho], error), flush=True)


        #reconstruction loop through light curves for every class other than rho              
        for n_sine, sine in enumerate(sine_file):
            valid_segments= sc.segmentation(sine, seg_len, seg_len , time_stamps=False)
            reco = sc.reconstruct(valid_segments, sine, cluster, rel_offset=False, seg_slide=seg_len)
            
            reco[0:-seg_len]=zscore(reco[0:-seg_len])
            sine_expected=zscore(np.copy((sine[0:-seg_len])))
            #sine_error=np.power(np.e,np.log(rho_expected)*0.5+1.0397207708265923)
            error = np.mean(((reco[0:-seg_len]-sine_expected))**2.0)
            reco_error.append((k_id,len_id,1,n_sine, error))
            print((k_id,len_id,1,n_sine, error), flush=True)
reco_error_ar=np.array(reco_error)
np.savetxt("chi2_20190610_6.csv", reco_error_ar, delimiter=",") 

(0, 0, 0, 993, 0.12598742201234925)
(0, 0, 0, 859, 0.09619052763557716)
(0, 0, 0, 298, 0.23796973114230807)
(0, 0, 0, 553, 0.1065803739745621)
(0, 0, 0, 672, 0.10509807674297761)
(0, 0, 0, 971, 0.1415131091587545)
(0, 0, 0, 27, 0.14286602124818235)
(0, 0, 0, 231, 0.15507256072411763)
(0, 0, 0, 306, 0.08518511106098976)
(0, 0, 0, 706, 0.09063476893630923)
(0, 0, 0, 496, 0.08861657027463443)
(0, 0, 0, 558, 0.20587410877031467)
(0, 0, 0, 784, 0.09216570827185358)
(0, 0, 0, 239, 0.08824209578858215)
(0, 0, 0, 578, 0.10999757974343666)
(0, 0, 0, 55, 0.2668740179473223)
(0, 0, 0, 906, 0.1819557696750518)
(0, 0, 0, 175, 0.13578281486381455)
(0, 0, 0, 14, 0.21198274988360882)
(0, 0, 0, 77, 0.16528249815214205)
(0, 0, 0, 31, 0.16419024454815967)
(0, 0, 0, 481, 0.0987590086742274)
(0, 0, 0, 310, 0.08964043997421645)
(0, 0, 0, 311, 0.12292959621551061)
(0, 0, 0, 883, 0.08961171168055955)
(0, 0, 0, 788, 0.10393305329720436)
(0, 0, 0, 45, 0.11652856593517481)
(0, 0, 0, 103, 0.1177065062500987)
(0, 